In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import pyarrow.compute as pc


In [2]:
speriod=int(input("Enter the simulation period: "))
samples=int(input("Enter the number of samples: "))

In [3]:
# Define the folder containing the Parquet files
folder_path = r'D:\RISHIN\13_ILC_TASK1\input\PARQUET_FILES'

# List all Parquet files in the folder
parquet_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]

In [93]:

# Check if there are any Parquet files in the folder
if parquet_files:
    # Read the first Parquet file in chunks
    parquet_file = pq.ParquetFile(parquet_files[0])
    for batch in parquet_file.iter_batches(batch_size=1000):
        # Convert the first batch to a PyArrow Table
        table = pa.Table.from_batches([batch])
        
        # Convert the PyArrow Table to a Pandas DataFrame
        df = table.to_pandas()
        
        # Extract the first value of LocationName and split it by '_'
        location_name = df['LocationName'].iloc[0]
        country = location_name.split('_')[0]
        
        # Ask user for output folder
        output_folder_path = input("Enter the output folder path: ")
        
        # Define the main folder path
        main_folder_path = os.path.join(output_folder_path, f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_Losses')
        
        # Define subfolders
        subfolders = ['EP', 'PLT', 'STATS']
        nested_folders = ['Lob', 'Portfolio']
        innermost_folders = ['GR', 'GU']
        
        # Create the main folder and subfolders
        for subfolder in subfolders:
            subfolder_path = os.path.join(main_folder_path, subfolder)
            os.makedirs(subfolder_path, exist_ok=True)
            
            for nested_folder in nested_folders:
                nested_folder_path = os.path.join(subfolder_path, nested_folder)
                os.makedirs(nested_folder_path, exist_ok=True)
                
                for innermost_folder in innermost_folders:
                    innermost_folder_path = os.path.join(nested_folder_path, innermost_folder)
                    os.makedirs(innermost_folder_path, exist_ok=True)
        
        print(f"Folders created successfully at {main_folder_path}")
        break  # Process only the first batch
else:
    print("No Parquet files found in the specified folder.")

Folders created successfully at D:\RISHIN\Rough\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses


In [ ]:
#lob name 

In [16]:


# Initialize an empty list to store the results
final_grouped_tables = []

# Process each Parquet file individually
for file in parquet_files:
    # Read the Parquet file into a PyArrow Table
    table = pq.read_table(file)
    
    # Perform the aggregation: sum the Loss column grouped by EventId, PeriodId, and LobName
    grouped_table = table.group_by(['EventId', 'PeriodId', 'LobName']).aggregate([('Loss', 'sum')])
    
    # Rename the aggregated column to Sum_Loss
    grouped_table = grouped_table.rename_columns(['EventId', 'PeriodId', 'LobName', 'Sum_Loss'])
    
    # Append the grouped Table to the final_grouped_tables list
    final_grouped_tables.append(grouped_table)

# Concatenate all grouped tables
final_table = pa.concat_tables(final_grouped_tables)

# Perform final grouping and sorting
final_grouped_table = final_table.group_by(['EventId', 'PeriodId', 'LobName']).aggregate([('Sum_Loss', 'sum')])
sorted_final_table = final_grouped_table.sort_by([('Sum_Loss_sum', 'descending')])
# The Table is now ready for the next instructions
dataframe_1 = sorted_final_table

In [19]:
dataframe_1= dataframe_1.to_pandas()


In [11]:
#dataframe_1 = dataframe_1[dataframe_1['LobName'] == 'AUTO']


In [39]:
# Initialize dataframe_2 by selecting PeriodId and max(Sum_Loss) grouped by PeriodId
dataframe_2 = dataframe_1.groupby(['PeriodId','LobName'], as_index=False).agg({'Sum_Loss_sum': 'max'})

# Rename the aggregated column to Max_Loss
dataframe_2.rename(columns={'Sum_Loss_sum': 'Max_Loss'}, inplace=True)

# Sort dataframe_2 by Max_Loss in descending order
dataframe_2 = dataframe_2.sort_values(by='Max_Loss', ascending=False)

# Initialize dataframe_2 by selecting PeriodId and Sum(Sum_Loss) grouped by PeriodId
dataframe_3 = dataframe_1.groupby(['PeriodId','LobName'], as_index=False).agg({'Sum_Loss_sum': 'sum'})

# Rename the aggregated column to Sum_Loss
dataframe_3.rename(columns={'Sum_Loss_sum': 'S_Sum_Loss'}, inplace=True)

# Sort dataframe_3 by S_sum_Loss in descending order
dataframe_3 = dataframe_3.sort_values(by='S_Sum_Loss', ascending=False)

#dataframe_2['Max_Loss'] = dataframe_2['Max_Loss'].round(5)

dataframe_2['rate'] = (1 / (speriod * samples))

# Calculate the cumulative rate column and round to 6 decimal places
dataframe_2['cumrate'] = dataframe_2['rate'].cumsum()

# Calculate the RPs column and round to 6 decimal places
dataframe_2['RPs'] = (1 / dataframe_2['cumrate'])




# Calculate the TCE_OEP_1 column and round to 6 decimal places
dataframe_2['TCE_OEP_1'] = ((dataframe_2['Max_Loss'] - dataframe_2['Max_Loss'].shift(-1)) * 
                          (dataframe_2['cumrate'] + dataframe_2['cumrate'].shift(-1)) * 0.5)

# Calculate the TCE_OEP_2 column and round to 6 decimal places
dataframe_2['TCE_OEP_2'] = (dataframe_2['TCE_OEP_1'].shift().cumsum() * dataframe_2['RPs'])


# Calculate the TCE_OEP_Final column and round to 6 decimal places
dataframe_2['TCE_OEP_Final'] = (dataframe_2['TCE_OEP_2'] + dataframe_2['Max_Loss'])

#dataframe_3['S_Sum_Loss'] = dataframe_3['S_Sum_Loss'].round(5)

# Calculate the rate column and round to 6 decimal places
dataframe_3['rate'] = (1 / (speriod * samples))

# Calculate the cumulative rate column and round to 6 decimal places
dataframe_3['cumrate'] = dataframe_3['rate'].cumsum()

# Calculate the RPs column and round to 6 decimal places
dataframe_3['RPs'] = (1 / dataframe_3['cumrate'])


# Calculate the TCE_AEP_1 column and round to 6 decimal places
dataframe_3['TCE_AEP_1'] = ((dataframe_3['S_Sum_Loss'] - dataframe_3['S_Sum_Loss'].shift(-1)) * 
                          (dataframe_3['cumrate'] + dataframe_3['cumrate'].shift(-1)) * 0.5)

# Calculate the cumulative sum up to the previous row and multiply by the current row's RPs, then round to 6 decimal places
dataframe_3['TCE_AEP_2'] = (dataframe_3['TCE_AEP_1'].shift().cumsum() * dataframe_3['RPs'])

# Calculate the TCE_AEP_Final column and round to 6 decimal places
dataframe_3['TCE_AEP_Final'] = (dataframe_3['TCE_AEP_2'] + dataframe_3['S_Sum_Loss'])





In [84]:
import numpy as np

# Define the list of RPs values to filter and convert them to float
rps_values = [float(x) for x in [10000, 5000, 1000, 500, 250, 200, 100, 50]]

# Define a tolerance for floating-point comparison
tolerance = 1e-6

# Initialize an empty DataFrame to store the filtered results
fdataframe_2 = pd.DataFrame()
fdataframe_3 = pd.DataFrame()

# Loop through each value in rps_values and filter the DataFrames
for value in rps_values:
    fdataframe_2 = pd.concat([fdataframe_2, dataframe_2[np.isclose(dataframe_2['RPs'], value, atol=tolerance)]])
    fdataframe_3 = pd.concat([fdataframe_3, dataframe_3[np.isclose(dataframe_3['RPs'], value, atol=tolerance)]])



In [85]:
fdataframe_3.rename(columns={'S_Sum_Loss': 'AEP','TCE_AEP_Final': 'TCE-AEP'}, inplace=True)
fdataframe_2.rename(columns={ 'Max_Loss': 'OEP','TCE_OEP_Final': 'TCE-OEP'}, inplace=True)



In [86]:
# Define the mapping of LobName to LobId
lobname_to_lobid = {
    'AGR': "1",
    'AUTO': "2",
    'COM': "3",
    'IND': "4",
    'SPER': "5",
    'FRST': "6",
    'GLH': "7"
}

# Add the LobId column to fdataframe_2
fdataframe_2['LobId'] = fdataframe_2['LobName'].map(lobname_to_lobid)

# Add the LobId column to fdataframe_3
fdataframe_3['LobId'] = fdataframe_3['LobName'].map(lobname_to_lobid)


In [89]:
# Define the columns to be used in the new DataFrame for fdataframe_3
columns_to_keep_3 = ['RPs', 'LobId', 'LobName']
columns_to_melt_3 = [ 'AEP','TCE-AEP']

# Melt fdataframe_3 to reshape it
melted_df_3 = fdataframe_3.melt(id_vars=columns_to_keep_3, value_vars=columns_to_melt_3, 
                                var_name='EPType', value_name='Loss')

# Rename columns to match the desired output
melted_df_3.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)

# Reorder columns
final_df_3 = melted_df_3[['EPType', 'Loss', 'ReturnPeriod', 'LobId', 'LobName']]

# Define the columns to be used in the new DataFrame for fdataframe_2
columns_to_keep_2 = ['RPs', 'LobId', 'LobName']
columns_to_melt_2 = [ 'OEP','TCE-OEP']

# Melt fdataframe_2 to reshape it
melted_df_2 = fdataframe_2.melt(id_vars=columns_to_keep_2, value_vars=columns_to_melt_2, 
                                var_name='EPType', value_name='Loss')

# Rename columns to match the desired output
melted_df_2.rename(columns={'RPs': 'ReturnPeriod'}, inplace=True)

# Reorder columns
final_df_2 = melted_df_2[['EPType', 'Loss', 'ReturnPeriod', 'LobId', 'LobName']]

# Concatenate the two DataFrames
final_df_EP_LOB_GU = pd.concat([final_df_3, final_df_2], ignore_index=True)



In [94]:
main_folder_path = os.path.join(output_folder_path, f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_Losses')

# Define the file path for the Parquet file
parquet_file_path = os.path.join(main_folder_path, 'EP', 'Lob', 'GU', f'ILC2024_EUWS_PLA_WI_EP_{country}_EUR_EP_Lob_GU_0.parquet')

# Save final_df as a Parquet file
final_df_EP_LOB_GU.to_parquet(parquet_file_path, index=False)

print(f"Parquet file saved successfully at {parquet_file_path}")

Parquet file saved successfully at D:\RISHIN\Rough\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\EP\Lob\GU\ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_0.parquet


In [96]:
import pandas as pd
import pyarrow.parquet as pq

# Define the path to the Parquet file
file_path = r'D:\RISHIN\Rough\ILC2024_EUWS_PLA_WI_EP_BE_EUR_Losses\EP\Lob\GU\ILC2024_EUWS_PLA_WI_EP_BE_EUR_EP_Lob_GU_0.parquet'

# Read the Parquet file into a PyArrow Table
table = pq.read_table(file_path)

# Convert the PyArrow Table to a Pandas DataFrame
df = table.to_pandas()



In [97]:
df

,EPType,Loss,ReturnPeriod,LobId,LobName
0,AEP,1.378475e+10,10000.0,5,SPER
1,AEP,1.072958e+10,5000.0,5,SPER
2,AEP,5.933762e+09,1000.0,5,SPER
3,AEP,4.504299e+09,500.0,5,SPER
4,AEP,3.098143e+09,250.0,5,SPER
5,AEP,2.774165e+09,200.0,5,SPER
6,AEP,1.890900e+09,100.0,5,SPER
7,AEP,1.263074e+09,50.0,5,SPER
8,TCE-AEP,1.770516e+10,10000.0,5,SPER
9,TCE-AEP,1.481754e+10,5000.0,5,SPER


In [66]:
# Display the schema (data types) of the DataFrame
print("Schema (data types):")
print(df.dtypes)

# Display the columns of the DataFrame
print("\nColumns:")
print(df.columns)

# Display the number of decimal places for floating-point numbers
print("\nDecimal places for floating-point numbers:")
for col in df.select_dtypes(include=['float64']).columns:
    decimal_places = df[col].apply(lambda x: len(str(x).split('.')[1]) if '.' in str(x) else 0).max()
    print(f"{col}: {decimal_places} decimal places")

# Display the first few rows of the DataFrame
print("\nFirst few rows of the DataFrame:")
print(df.head())

Schema (data types):
EPType           object
Loss            float64
ReturnPeriod    float64
LobId            object
LobName          object
dtype: object

Columns:
Index(['EPType', 'Loss', 'ReturnPeriod', 'LobId', 'LobName'], dtype='object')

Decimal places for floating-point numbers:
Loss: 10 decimal places
ReturnPeriod: 1 decimal places

First few rows of the DataFrame:
  EPType          Loss  ReturnPeriod LobId LobName
0    OEP  8.639885e+08       10000.0     2    AUTO
1    OEP  6.749918e+08        5000.0     2    AUTO
2    OEP  3.778001e+08        1000.0     2    AUTO
3    OEP  2.847455e+08         500.0     2    AUTO
4    OEP  2.064693e+08         250.0     2    AUTO
